In [1]:
import pandas
reviews = pandas.read_csv("data/denormalized-moderated-2021.csv")
reviews['label'] = reviews.apply(lambda row: 1 if row['Accept/Reject'] == 'Accept' else 0, axis=1)
field_separator = "\n"
reviews['full_text'] = reviews.apply(lambda row: f"{row['screenname']}{field_separator}{row['title']}{field_separator}{row['comments']}{field_separator}{row['exterior_review']}{field_separator}{row['interior_review']}{field_separator}{row['family_review']}{field_separator}{row['cost_review']}{field_separator}{row['perf_review']}", axis=1)
reviews[['label','full_text']].head()


,label,full_text
0,0,Jose\nGreat love it\nI only buy toyota they do...
1,0,Subaru Fan\nHardly been driven\nBeen the sport...
2,1,WDS\nFeatures\nThese are the features I like o...
3,1,"Nino\nGreat car!\nGreat, fun, energetic car! I..."
4,1,BMW650IGranCoupe\nTrue performance \nThis car ...


In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [3]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 128.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 KB 38.0 MB/s eta 0:00:00
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
def preprocess_function(row):
    return tokenizer(str(row['full_text']), truncation=True, padding=True)

In [6]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached datasets-2.7.0-py3-none-any.whl (451 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 KB 2.2 MB/s eta 0:00:00a 0:00:01
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [ ]:

# reviews = pandas.DataFrame(data={'full_text': ["great car", 
#                                         "fantastic ride",
#                                         "has some issues",
#                                        "best car ever",
#                                        "sucks blows horrible",
#                                        "mamba buahaha",
#                                        "good car",
#                                         "my best ride",
#                                         "your mama is so fat",
#                                         "bring back the manuals",
#                                         "honda rocks",
#                                         "my honda drives best after it had a few drinks",
#                                         "good bad and the ugly about this car",
#                                         "is not a good car",
#                                         "i like to read books and long walks on the beach",
#                                         "your butt smells"
#                                 ],
#                           'label': [1, 1,1,1,0,0,1,1,0,1,1,0,1,1,0,0]})

In [7]:
train = reviews.sample(frac=0.8, random_state= 25)
test = reviews.drop(train.index)

from datasets import Dataset
train_ds = Dataset.from_pandas(train[['full_text','label']])
test_ds = Dataset.from_pandas(test[['full_text','label']])

tokenized_train_ds = train_ds.map(preprocess_function)
tokenized_test_ds = test_ds.map(preprocess_function)

  0%|          | 0/98867 [00:00<?, ?ex/s]

  0%|          | 0/24717 [00:00<?, ?ex/s]

In [8]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [34]:
training_args = TrainingArguments(
    output_dir="./results2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 98867
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 92700
  Number of trainable parameters = 66955010
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: full_text, __index_level_0__. If full_text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.


Step,Training Loss
500,0.342200
1000,0.326100
1500,0.308600
2000,0.311200
2500,0.300200
3000,0.312400
3500,0.294600
4000,0.306600
4500,0.300000
5000,0.301600


Saving model checkpoint to ./results2/checkpoint-500
Configuration saved in ./results2/checkpoint-500/config.json
Model weights saved in ./results2/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results2/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results2/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results2/checkpoint-1000
Configuration saved in ./results2/checkpoint-1000/config.json
Model weights saved in ./results2/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results2/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results2/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results2/checkpoint-1500
Configuration saved in ./results2/checkpoint-1500/config.json
Model weights saved in ./results2/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results2/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results2/checkpoint-15

In [15]:
import torch
torch.cuda.empty_cache()

#print(torch.cuda.is_available())
#torch.cuda.device_count()
#torch.cuda.get_device_name(0)

In [33]:
reviews = None
test = None
train = None
train_ds = None
test_ds = None

import gc
gc.collect()

471

In [21]:
!nvidia-smi

Mon Nov 21 20:19:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   23C    P0    59W / 300W |  22661MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------